In [1]:
import numpy as np
import tensorflow as tf
!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.0-rc1
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGBRYFH/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGBRYFH/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGBRYFH/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [4]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [5]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=40,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 2s 69ms/step - loss: 0.8114 - accuracy: 0.5314 - val_loss: 0.7005 - val_accuracy: 0.5900
Epoch 2/40
30/30 [==============================] - 2s 65ms/step - loss: 0.6529 - accuracy: 0.6184 - val_loss: 0.6109 - val_accuracy: 0.6569
Epoch 3/40
30/30 [==============================] - 2s 65ms/step - loss: 0.5822 - accuracy: 0.6755 - val_loss: 0.5630 - val_accuracy: 0.7008
Epoch 4/40
30/30 [==============================] - 2s 64ms/step - loss: 0.5380 - accuracy: 0.7085 - val_loss: 0.5283 - val_accuracy: 0.7133
Epoch 5/40
30/30 [==============================] - 2s 64ms/step - loss: 0.5013 - accuracy: 0.7401 - val_loss: 0.4986 - val_accuracy: 0.7448
Epoch 6/40
30/30 [==============================] - 2s 64ms/step - loss: 0.4662 - accuracy: 0.7687 - val_loss: 0.4696 - val_accuracy: 0.7545
Epoch 7/40
30/30 [==============================] - 2s 65ms/step - loss: 0.4323 - accuracy: 0.7932 - val_loss: 0.4421 - val_accuracy: 0.7772
Epoch 8/40
30

In [7]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.4461 - accuracy: 0.8514
loss: 0.446
accuracy: 0.851
